In [1]:
## Pricing in Options and their epected move 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import sqlite3 as sql 
import datetime as dt 
print(dt.datetime.now())

2024-03-01 12:05:16.971214


In [3]:
options_file_path = '../../../../data/options/options.db'
price_file_path = '../../../../data/prices/stocks.db'

option_db = sql.connect(options_file_path)
price_db = sql.connect(price_file_path)

In [7]:
stock = 'ttd'
stock = 'roku'
lim = 3

def exp_moves_w_contracts(stock, option_db=option_db, price_db=price_db, lim = 3):
    exp_move = f'''
        -- Select the maximum strike for ITM calls 
        with calls as (
            select
                expiry, 
                stk_price,
                max(strike) as call_strike,
                ask as call_ask
            from
                {stock}
            where
                date(gatherdate) = (select max(date(gatherdate)) from {stock})
            and
                type = 'Call'
            and 
                stk_price > strike
            group by expiry) 
        , 
        puts as (
            select 
                expiry, 
                min(strike) as put_strike,
                ask as put_ask
            from
                {stock}
            where
                datetime(gatherdate) = (select max(datetime(gatherdate)) from {stock})
            and
                type = 'Put'
            and
                stk_price < strike
            group by expiry)

        select 
            calls.expiry
            ,calls.stk_price
            --,calls.call_ask
            --,puts.put_strike
            --,puts.put_ask
            ,(calls.call_ask + puts.put_ask) * 0.85 as exp_move
            ,concat(
                concat("±", round(100 * (((calls.call_ask + puts.put_ask) * 0.85) / calls.stk_price), 2)), "%")
                as "%"
        from
            calls
        join
            puts
        on
            calls.expiry = puts.expiry
        where 
            date(calls.expiry) > date('now')
        and 
            date(puts.expiry) > date('now')
        order by 
            date(calls.expiry)
        limit {lim}

        '''

    d = pd.read_sql(exp_move, option_db, parse_dates = ['expiry'])
    d.insert(1, 'stock', stock.upper())

    d['1𝝈'] = list(zip((d.stk_price + d.exp_move).round(2), (d.stk_price - d.exp_move).round(2)))
    d['2𝝈'] = list(zip((d.stk_price + 2 * d.exp_move).round(2), (d.stk_price - 2 * d.exp_move).round(2)))

    call_strikes = [(x[0], y[0]) for x,y  in zip(list(d['1𝝈' ]), list(d['2𝝈']))]
    put_strikes = [(x[1], y[1]) for x,y  in zip(list(d['1𝝈' ]), list(d['2𝝈']))]
    exps = d.expiry.to_list()

    # Get All strikes for the expiration date
    all_strikes = pd.read_sql(f'''
        select 
            distinct strike 
        from {stock} 
        where 
            datetime(gatherdate) = (select max(datetime(gatherdate)) from {stock}) and date(expiry) = date("{exps[2]}")
    ''', option_db)
    numbers = sorted(list(all_strikes.strike))
    cs_1sd = [call_strikes[x][0] for x in range(len(call_strikes))]
    cs_2sd = [call_strikes[x][1] for x in range(len(call_strikes))]
    ps_1sd = [put_strikes[x][0] for x in range(len(put_strikes))]
    ps_2sd = [put_strikes[x][1] for x in range(len(put_strikes))]

    cs = [tuple((min(numbers, key=lambda x:abs(x-cs_1sd[j])),min(numbers, key=lambda x:abs(x-cs_2sd[j])))) for j in range(len(cs_1sd))]
    ps = [tuple((min(numbers, key=lambda x:abs(x-ps_1sd[j])),min(numbers, key=lambda x:abs(x-ps_2sd[j])))) for j in range(len(ps_1sd))]

    # Get the Prices for the Strikes 
    call_strikes = [item for sublist in cs for item in sublist]
    put_strikes = [item for sublist in ps for item in sublist]
    avail_strikes = f'''
    select 
        date(expiry) as expiry
        ,type
        ,strike
        ,ask
        ,case when type = 'Call' then (((strike + ask) - stk_price) / stk_price ) * 100
            else 100 *(((strike - ask) - stk_price) / stk_price) end as "%OTM"
    from 
        {stock}
    where
        datetime(gatherdate) = (select max(datetime(gatherdate)) from {stock})
    and
        date(expiry) = date("{exps[1]}")
    and 
        case when type = 'Call' then strike in {cs[0]}
            else strike in {ps[0]} end
    order by 
        type, strike
    '''
    c = pd.read_sql(avail_strikes, option_db)
    d['Call Strikes'] = sorted(cs)
    d['Puts Strikes'] = sorted(ps)
    call_prices = c[c.type == 'Call'].ask.to_list()
    put_prices = c[c.type == 'Put'].ask.to_list()
    call_be = c[c.type == 'Call']["%OTM"].div(100).round(2).to_list()
    put_be = c[c.type == 'Put']["%OTM"].div(100).round(2).to_list()

    d['Call_Prices'] = '('+ str(call_prices[0]) + ', ' + str(call_prices[1]) + ')'
    d['Call_BE%'] = '('+ str(call_be[0]) + ', ' + str(call_be[1]) + ')'
    d['Put_Prices'] = '('+ str(put_prices[1]) + ', ' + str(put_prices[0]) + ')'
    d['Put_BE%'] = '('+ str(put_be[1]) + ', ' + str(put_be[0]) + ')'
    #return d.iloc[0].to_frame().T
    return d

exp_moves_w_contracts('sq', option_db, price_db, lim = 3)

IndexError: list index out of range

In [5]:
from tqdm import tqdm 
stocks = ['lcid','mos','arm','amd']
lodf = []
contract_list = {}
for x in tqdm(stocks):
    try: 
        df = exp_moves_w_contracts(x, option_db, price_db, lim = 2)
        lodf.append(df.iloc[-1].to_frame().T)
    except:
        print(f'Error with {x}')
        continue
    
pd.concat(lodf)

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:01<00:04,  1.40s/it]

Error with lcid


 50%|█████     | 2/4 [00:02<00:02,  1.32s/it]

Error with mos


 75%|███████▌  | 3/4 [00:02<00:00,  1.23it/s]

Error with arm


100%|██████████| 4/4 [00:05<00:00,  1.28s/it]

Error with amd


ValueError: No objects to concatenate

In [8]:
from IPython.display import display

def exp_moves_w_contracts(stock, option_db=option_db, price_db=price_db, lim = 3):
    exp_move = f'''
        -- Select the maximum strike for ITM calls 
        with calls as (
            select
                expiry, 
                stk_price,
                max(strike) as call_strike,
                ask as call_ask
            from
                {stock}
            where
                date(gatherdate) = (select max(date(gatherdate)) from {stock})
            and
                type = 'Call'
            and 
                stk_price > strike
            group by expiry) 
        , 
        puts as (
            select 
                expiry, 
                min(strike) as put_strike,
                ask as put_ask
            from
                {stock}
            where
                datetime(gatherdate) = (select max(datetime(gatherdate)) from {stock})
            and
                type = 'Put'
            and
                stk_price < strike
            group by expiry)

        select 
            calls.expiry
            ,calls.stk_price
            --,calls.call_ask
            --,puts.put_strike
            --,puts.put_ask
            ,(calls.call_ask + puts.put_ask) * 0.85 as exp_move
            ,concat(
                concat("±", round(100 * (((calls.call_ask + puts.put_ask) * 0.85) / calls.stk_price), 2)), "%")
                as "%"
        from
            calls
        join
            puts
        on
            calls.expiry = puts.expiry
        where 
            date(calls.expiry) > date('now')
        and 
            date(puts.expiry) > date('now')
        order by 
            date(calls.expiry)
        limit {lim if lim > 1 else 2}

        '''

    d = pd.read_sql(exp_move, option_db, parse_dates = ['expiry'])
    d.insert(1, 'stock', stock.upper())
    d['1𝝈'] = list(zip((d.stk_price + d.exp_move).round(2), (d.stk_price - d.exp_move).round(2)))
    d['2𝝈'] = list(zip((d.stk_price + 2 * d.exp_move).round(2), (d.stk_price - 2 * d.exp_move).round(2)))

    call_strikes = [(x[0], y[0]) for x,y  in zip(list(d['1𝝈' ]), list(d['2𝝈']))]
    put_strikes = [(x[1], y[1]) for x,y  in zip(list(d['1𝝈' ]), list(d['2𝝈']))]
    exps = d.expiry.to_list()

    # Get All strikes for the expiration date
    all_strikes = pd.read_sql(f'''
        select 
            distinct strike 
        from {stock} 
        where 
            datetime(gatherdate) = (select max(datetime(gatherdate)) from {stock}) and date(expiry) = date("{exps[1]}")
        ''', option_db)
    
    numbers = sorted(list(all_strikes.strike))
    cs_1sd = [call_strikes[x][0] for x in range(len(call_strikes))]
    cs_2sd = [call_strikes[x][1] for x in range(len(call_strikes))]
    ps_1sd = [put_strikes[x][0] for x in range(len(put_strikes))]
    ps_2sd = [put_strikes[x][1] for x in range(len(put_strikes))]
    
    print(cs_1sd, cs_2sd, ps_1sd, ps_2sd)

    cs = [tuple((min(numbers, key=lambda x:abs(x-cs_1sd[j])),min(numbers, key=lambda x:abs(x-cs_2sd[j])))) for j in range(len(cs_1sd))]
    ps = [tuple((min(numbers, key=lambda x:abs(x-ps_1sd[j])),min(numbers, key=lambda x:abs(x-ps_2sd[j])))) for j in range(len(ps_1sd))]

    # Get the Prices for the Strikes 
    call_strikes = [item for sublist in cs for item in sublist]
    put_strikes = [item for sublist in ps for item in sublist]
    
    avail_strikes = f'''
    select 
        date(expiry) as expiry
        ,type
        ,cast(strike as float) as strike
        ,ask
        ,case when type = 'Call' then (((strike + ask) - stk_price) / stk_price ) 
            else (((strike - ask) - stk_price) / stk_price) end as "OTM"
    from 
        {stock}
    where
        datetime(gatherdate) = (select max(datetime(gatherdate)) from {stock})
    and
        date(expiry) = date("{exps[1]}")
    and 
        case when type = 'Call' then strike in {tuple([x[0] for x in sorted(cs)]) + tuple([x[1] for x in sorted(cs)])}
            else strike in {tuple([x[0] for x in sorted(ps)]) + tuple([x[1] for x in sorted(ps)])} end
    order by 
        type, strike
    ''' 
    
    c = pd.read_sql(avail_strikes, option_db)
    # d[('Call','1𝝈','Strike')] = [x[0] for x in sorted(cs)]
    # d[('Call','1𝝈','Ask')] = c[c['strike'].isin([x[0] for x in sorted(cs)])]['ask'].to_list()  
    # d[('Call','1𝝈','BE')] = c[c['strike'].isin([x[0] for x in sorted(cs)])]['OTM'].to_list()  
    
    # d[('Put','1𝝈','Put Strike')] = [x[0] for x in sorted(ps)]
    # d[('Put','1𝝈','Put Ask')] = c[c['strike'].isin([x[0] for x in sorted(ps)])]['ask'].to_list()
    # d[('Put','1𝝈','Put BE')] = c[c['strike'].isin([x[0] for x in sorted(ps)])]['OTM'].to_list()
    
    # d[('Call','2𝝈','Strike')] = [x[1] for x in sorted(cs)]
    # d[('Call','2𝝈','Ask')] = c[c['strike'].isin([x[1] for x in sorted(cs)])]['ask'].to_list()
    # d[('Call','2𝝈','BE')] = c[c['strike'].isin([x[1] for x in sorted(cs)])]['OTM'].to_list()
        
    # d[('Put','2𝝈','Put Strike')] = [x[1] for x in sorted(ps)]
    # d[('Put','2𝝈','Put Ask')] = c[c['strike'].isin([x[1] for x in sorted(ps)])]['ask'].to_list()
    # d[('Put','2𝝈','Put BE')] = c[c['strike'].isin([x[1] for x in sorted(ps)])]['OTM'].to_list()
    
    ######################################################
    display(c)
    display(d)

    return d

d = exp_moves_w_contracts('snap', option_db, price_db, lim = 3)

[11.17, 11.17, 11.17] [11.17, 11.17, 11.17] [11.17, 11.17, 11.17] [11.17, 11.17, 11.17]


,expiry,type,strike,ask,OTM
0,2024-03-15,Call,11.0,0.0,-0.023957
1,2024-03-15,Put,11.0,0.0,-0.023957


,expiry,stock,stk_price,exp_move,%,1𝝈,2𝝈
0,2024-03-08,SNAP,11.17,0.0,±0.0%,"(11.17, 11.17)","(11.17, 11.17)"
1,2024-03-15,SNAP,11.17,0.0,±0.0%,"(11.17, 11.17)","(11.17, 11.17)"
2,2024-03-22,SNAP,11.17,0.0,±0.0%,"(11.17, 11.17)","(11.17, 11.17)"


In [65]:
em = d.set_index(['stock', 'expiry', 'stk_price', 'exp_move',  '1𝝈', '2𝝈', '%']).copy()

# MultiIndex Columns 
em.columns = pd.MultiIndex.from_tuples(em.columns)
em

Call  \
                                                                                 1𝝈   
                                                                             Strike   
stock expiry     stk_price exp_move 1𝝈               2𝝈               %               
BA    2024-03-01 201.73    6.290    (208.02, 195.44) (214.31, 189.15) ±3.12%  207.5   
      2024-03-08 201.73    8.585    (210.32, 193.14) (218.9, 184.56)  ±4.26%  210.0   

                                                                                    \
                                                                                     
                                                                               Ask   
stock expiry     stk_price exp_move 1𝝈               2𝝈               %              
BA    2024-03-01 201.73    6.290    (208.02, 195.44) (214.31, 189.15) ±3.12%  2.14   
      2024-03-08 201.73    8.585    (210.32, 193.14) (218.9, 184.56)  ±4.26%  1.53   

                                                                                        \
                                                                                         
                                                                                    BE   
stock expiry     stk_price exp_move 1𝝈               2𝝈               %                  
BA    2024-03-01 201.73    6.290    (208.02, 195.44) (214.31, 189.15) ±3.12%  0.039211   
      2024-03-08 201.73    8.585    (210.32, 193.14) (218.9, 184.56)  ±4.26%  0.048580   

                                                                                    Put  \
                                                                                     1𝝈   
                                                                             Put Strike   
stock expiry     stk_price exp_move 1𝝈               2𝝈               %                   
BA    2024-03-01 201.73    6.290    (208.02, 195.44) (214.31, 189.15) ±3.12%      192.5   
      2024-03-08 201.73    8.585    (210.32, 193.14) (218.9, 184.56)  ±4.26%      195.0   

                                                                                      \
                                                                                       
                                                                             Put Ask   
stock expiry     stk_price exp_move 1𝝈               2𝝈               %                
BA    2024-03-01 201.73    6.290    (208.02, 195.44) (214.31, 189.15) ±3.12%    1.28   
      2024-03-08 201.73    8.585    (210.32, 193.14) (218.9, 184.56)  ±4.26%    1.84   

                                                                                        \
                                                                                         
                                                                                Put BE   
stock expiry     stk_price exp_move 1𝝈               2𝝈               %                  
BA    2024-03-01 201.73    6.290    (208.02, 195.44) (214.31, 189.15) ±3.12% -0.052099   
      2024-03-08 201.73    8.585    (210.32, 193.14) (218.9, 184.56)  ±4.26% -0.042483   

                                                                               Call  \
                                                                                 2𝝈   
                                                                             Strike   
stock expiry     stk_price exp_move 1𝝈               2𝝈               %               
BA    2024-03-01 201.73    6.290    (208.02, 195.44) (214.31, 189.15) ±3.12%  215.0   
      2024-03-08 201.73    8.585    (210.32, 193.14) (218.9, 184.56)  ±4.26%  220.0   

                                                                                    \
                                                                                     
                                                                               Ask   
stock expiry     stk_price exp_move 1𝝈               2𝝈               %              
BA    2024-03-

In [66]:
from tqdm import tqdm 
import json 

st = '../../data/stocks/tickers.json'
stocks = json.load(open(st))['all_stocks']

lodf = []
contract_list = {}
error = []
for x in tqdm(stocks):
    try: 
        df = exp_moves_w_contracts(x, option_db, price_db, ).head(1)
        lodf.append(df)
    except:
        error.append(x)
        continue
    
allem = pd.concat(lodf)

100%|██████████| 172/172 [04:40<00:00,  1.63s/it]


In [68]:
print(error)

['gdx', 'gme', 'amzn', 'spy', 'xlf', 'aapl', 'pfe', 'aal', 'igv', 'uber', 'xli', 'uup', 'pep', 'mmm', 'ntr', 'uwmc', 'xlu', 'ups', 'pltr', 'sq', 'vxx', 'wmt', 'cvx', 'fcel', 'vale', 'mo', 'smh', 'hd', 'shy', 'rrc', 'fdn', 'lc', 'jks', 'vuzi', 'bkkt', 'bno', 'lmt', 'wfc', 'mrk', 'slv', 'qcom', 'sol', 'ardx', 'mos', 'qs', 'tgt', 'abbv', 'ocn', 'moo', 'rkt', 'iei', 'pbw', 'ccj', 'xme', 'ebay', 'f', 'ief', 'sono', 'pypl', 'ibm', 'amc', 'xom', 'xlk', 'xlre', 'spi', 'xlv', 'bito', 'ibb', 'sbux', 'jets', 'tlt', 'xlc', 'hsbc', 'kr', 'rtx', 'gsl', 'wba', 'ung', 'ba', 'ge', 'xrt', 'snap', 'ctva', 'corn', 'csco', 'cvs', 'chgg', 'ko', 'icln', 'snow', 'fang', 'li', 'run', 'tsn', 'iwm', 'spwr', 'lulu', 'itb', 'oxy', 'tbt', 'c', 'gld', 'lcid', 'weat', 'v', 'x', 'tan', 'clx', 'jpm', 'k', 'dcfc', 'tmv', 'sony', 'hood', 'pho', 'cag', 'uvxy', 'bil', 'ual', 'pgr', 'afl', 'aig', 'sofi', 'mpw', 'amat', 'dash', 'open', 'dbx', 'panw', 'tdoc']


In [118]:
q = '''
    select 
        date(expiry) as expiry, 
        type, 
        strike 
    from
        arm 
    where 
        date(gatherdate) = (select max(date(gatherdate)) from arm)
    and 
        date(expiry) = date("2024-04-05")
    order by 
        cast(strike as float)

'''

pd.read_sql(q, option_db).strike.max()

139.0